## Linear regression using PyTorch built-ins



In [7]:
import torch.nn as nn
import numpy as np
import torch

In [8]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [9]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

## Dataset and DataLoader



In [10]:
from torch.utils.data import TensorDataset

In [11]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [12]:
from torch.utils.data import DataLoader

In [13]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [14]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]])
tensor([[ 22.,  37.],
        [ 22.,  37.],
        [119., 133.],
        [103., 119.],
        [119., 133.]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [15]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2971,  0.5344,  0.2194],
        [ 0.0758,  0.5278, -0.0901]], requires_grad=True)
Parameter containing:
tensor([ 0.4680, -0.4246], requires_grad=True)


In [16]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2971,  0.5344,  0.2194],
         [ 0.0758,  0.5278, -0.0901]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4680, -0.4246], requires_grad=True)]

In [17]:
# Generate predictions
preds = model(inputs)
preds

tensor([[24.0179, 36.5932],
        [34.4995, 47.1476],
        [58.9521, 71.6624],
        [ 1.2617, 26.6661],
        [46.6263, 49.1611],
        [24.0179, 36.5932],
        [34.4995, 47.1476],
        [58.9521, 71.6624],
        [ 1.2617, 26.6661],
        [46.6263, 49.1611],
        [24.0179, 36.5932],
        [34.4995, 47.1476],
        [58.9521, 71.6624],
        [ 1.2617, 26.6661],
        [46.6263, 49.1611]], grad_fn=<AddmmBackward>)

## Loss Function



In [18]:
# Import nn.functional
import torch.nn.functional as F

In [19]:
# Define loss function
loss_fn = F.mse_loss

In [20]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(2316.1611, grad_fn=<MseLossBackward>)


## Optimizer



In [21]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model



1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero



In [22]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [26]:
fit(500, model, loss_fn, opt,train_dl)

Epoch [10/500], Loss: 1.2023
Epoch [20/500], Loss: 1.4953
Epoch [30/500], Loss: 1.7108
Epoch [40/500], Loss: 1.5098
Epoch [50/500], Loss: 1.5791
Epoch [60/500], Loss: 0.9936
Epoch [70/500], Loss: 1.2201
Epoch [80/500], Loss: 1.9207
Epoch [90/500], Loss: 1.1458
Epoch [100/500], Loss: 1.8183
Epoch [110/500], Loss: 1.7560
Epoch [120/500], Loss: 0.9448
Epoch [130/500], Loss: 1.6407
Epoch [140/500], Loss: 1.3002
Epoch [150/500], Loss: 1.1400
Epoch [160/500], Loss: 0.8778
Epoch [170/500], Loss: 0.9771
Epoch [180/500], Loss: 0.8428
Epoch [190/500], Loss: 1.3210
Epoch [200/500], Loss: 0.9793
Epoch [210/500], Loss: 0.9421
Epoch [220/500], Loss: 0.9625
Epoch [230/500], Loss: 0.8100
Epoch [240/500], Loss: 0.9760
Epoch [250/500], Loss: 0.8266
Epoch [260/500], Loss: 0.8374
Epoch [270/500], Loss: 1.2187
Epoch [280/500], Loss: 0.8023
Epoch [290/500], Loss: 0.7863
Epoch [300/500], Loss: 0.8730
Epoch [310/500], Loss: 0.7184
Epoch [320/500], Loss: 0.7618
Epoch [330/500], Loss: 0.9098
Epoch [340/500], Lo

In [27]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.2599,  70.2594],
        [ 82.0632, 100.4320],
        [118.8977, 133.5092],
        [ 21.1327,  37.1713],
        [101.7309, 118.5543],
        [ 57.2599,  70.2594],
        [ 82.0632, 100.4320],
        [118.8977, 133.5092],
        [ 21.1327,  37.1713],
        [101.7309, 118.5543],
        [ 57.2599,  70.2594],
        [ 82.0632, 100.4320],
        [118.8977, 133.5092],
        [ 21.1327,  37.1713],
        [101.7309, 118.5543]], grad_fn=<AddmmBackward>)

In [28]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])